In [3]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator #Augmentation
from keras.callbacks import EarlyStopping

In [ ]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0,1,2,3))
        X_train = (X_train-mean)/(std+1e-7)  #1e-7為1*10的-7次方
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test
    
    
## Normalize Training and Testset    
x_train, x_test = normalize(x_train, x_test) 

## OneHot Label 由(None, 1)-(None, 10)
one_hot = OneHotEncoder()
y_train = one_hot.fit_transform(y_train).toarray()
y_test = one_hot.transform(y_test).toarray()


classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,3,3, input_shape=(32,32,3), activation='relu'))
classifier.add(BatchNormalization())   # BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32,3,3, activation='relu'))
classifier.add(BatchNormalization())  # BatchNormalization
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100, activation='relu',
                     kernel_regularizer=regularizers.l2(l=0.001))) ##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(p=0.5)) ##Dropout

classifier.add(Dense(output_dim=100,activation='relu',
                     kernel_regularizer=regularizers.l2(0.001)))##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization

classifier.add(Dropout(p=0.3))##Dropout

classifier.add(Dense(output_dim=10,activation='softmax'))
#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])



img_gen = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,
                             rotation_range=10,width_shift_range=0.1,
                             height_shift_range=0.1,shear_range=0.1,zoom_range=0.1,
                             horizontal_flip=True,vertical_flip=False,dtype=np.float32)
img_gen.fit(x_train)



earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) #earlystop

# 開始訓練
classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=100) ,steps_per_epoch=500,
                         epochs=100, validation_data = (x_test, y_test),
                         callbacks = [earlystop])

(50000, 32, 32, 3)


<ipython-input-4-18fe8471c089>:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu")`
  classifier.add(Convolution2D(32,3,3, input_shape=(32,32,3), activation='relu'))
<ipython-input-4-18fe8471c089>:31: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
  classifier.add(Convolution2D(32,3,3, activation='relu'))
<ipython-input-4-18fe8471c089>:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
  classifier.add(Dense(output_dim=100, activation='relu',
<ipython-input-4-18fe8471c089>:42: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`
  classifier.add(Dropout(p=0.5)) ##Dropout
<ipython-input-4-18fe8471c089>:44: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
  classifier.add(D

Epoch 1/100
500/500 [==============================] - 84s 169ms/step - loss: 2.1137 - accuracy: 0.3576 - val_loss: 1.6564 - val_accuracy: 0.5059
Epoch 2/100
  1/500 [..............................] - ETA: 1:21 - loss: 1.8084 - accuracy: 0.4500

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/callbacks/callbacks.py:843: RuntimeWarning: Early stopping conditioned on metric `test_loss` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  warnings.warn(


500/500 [==============================] - 98s 197ms/step - loss: 1.6752 - accuracy: 0.4798 - val_loss: 1.4768 - val_accuracy: 0.5503
Epoch 3/100
322/500 [==================>...........] - ETA: 42s - loss: 1.5262 - accuracy: 0.5275